In [7]:
!python main.py

❌ Failed to fetch item BXV7FDKRLGZBJOE5PVBXBFYUXNSL3UVS-ocr_1_DDB_FULLTEXT
Skipping item BXV7FDKRLGZBJOE5PVBXBFYUXNSL3UVS-ocr_1_DDB_FULLTEXT due to a download error.
❌ Failed to fetch item BXV7FDKRLGZBJOE5PVBXBFYUXNSL3UVS-ocr_2_DDB_FULLTEXT
Skipping item BXV7FDKRLGZBJOE5PVBXBFYUXNSL3UVS-ocr_2_DDB_FULLTEXT due to a download error.
❌ Failed to fetch item BXV7FDKRLGZBJOE5PVBXBFYUXNSL3UVS-ocr_3_DDB_FULLTEXT
Skipping item BXV7FDKRLGZBJOE5PVBXBFYUXNSL3UVS-ocr_3_DDB_FULLTEXT due to a download error.
❌ Failed to fetch item BXV7FDKRLGZBJOE5PVBXBFYUXNSL3UVS-ocr_4_DDB_FULLTEXT
Skipping item BXV7FDKRLGZBJOE5PVBXBFYUXNSL3UVS-ocr_4_DDB_FULLTEXT due to a download error.

📥 Downloading XMLs to: ddb\BXV7FDKRLGZBJOE5PVBXBFYUXNSL3UVS
✔ Saved ddb\BXV7FDKRLGZBJOE5PVBXBFYUXNSL3UVS\page_1.xml
✔ Saved ddb\BXV7FDKRLGZBJOE5PVBXBFYUXNSL3UVS\page_2.xml
✔ Saved ddb\BXV7FDKRLGZBJOE5PVBXBFYUXNSL3UVS\page_3.xml
✔ Saved ddb\BXV7FDKRLGZBJOE5PVBXBFYUXNSL3UVS\page_4.xml
Processing item BXV7FDKRLGZBJOE5PVBXBFYUXNSL3UVS, c